このノートブックでは、インピュテーションサーバの出力データを用いて、PRS の計算手順を説明します。
# Step 1 hail など、必要なモジュールを読み込みます
下記のコードを実行してください。 ページ上側のメニューバーにある 実行 ボタンを押下することで、実行することができます。

In [1]:
import hail as hl
hl.init()
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2022-09-13 16:56:27 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://b45c895afb21:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /notebooks/hail-20220913-1656-0.2.97-937922d7f46c.log


Loading BokehJS ...

# Step 2 ゲノムデータを読み込みます
下記のコードでは、一般的なゲノムデータのファイル形式である `VCF` フォーマット（`outputs/chr1.beagle.vcf.gz`）から `hail` のファイル形式である `matrix table`フォーマット (`outputs/chr1.beagle.mt`) に変換します。

下記のコードを実行した後、1分程度の待ち時間が生じます。


In [4]:
hl.import_vcf('outputs/chr1.beagle.vcf.gz', force_bgz=True).write('outputs/chr1.beagle.mt', overwrite=True)

2022-09-13 17:02:01 Hail: INFO: Coerced sorted dataset=====>        (6 + 1) / 7]
2022-09-13 17:07:25 Hail: INFO: wrote matrix table with 2428653 rows and 2318 columns in 7 partitions to outputs/chr1.beagle.mt


`mt` というオブジェクトにゲノムデータを読み込みます。 なお、`mt` は `matrix table` の略です。


In [5]:
mt = hl.read_matrix_table('outputs/chr1.beagle.mt')

読み込んだゲノムデータに含まれる研究対象者の人数とバリアントの個数を表示します。

In [6]:
mt.count()

(2428653, 2318)

(2428653, 2318)

と表示されました。

これは、次のことを意味します。

- 研究対象者の人数が 2318 名
- バリアントの個数が 2428,653 個


# Step 3 ゲノムデータに variantID を追加します
下記のコードは、読み込まれたゲノムデータのバリアント情報を表示します。

show(5) は、先頭の 5 個のバリアントのみを表示する、という意味です。

In [9]:
mt.rows().show(5)

+---------------+--------------------------------+---------------+-----------+
| locus         | alleles                        | rsid          |      qual |
+---------------+--------------------------------+---------------+-----------+
| locus<GRCh37> | array<str>                     | str           |   float64 |
+---------------+--------------------------------+---------------+-----------+
| 1:10177       | ["A","AC"]                     | "rs367896724" | -1.00e+01 |
| 1:10235       | ["T","TA"]                     | "rs540431307" | -1.00e+01 |
| 1:10352       | ["T","TA"]                     | "rs555500075" | -1.00e+01 |
| 1:10616       | ["CCGCCGTTGCAAAGGCGCGCCG","C"] | "rs376342519" | -1.00e+01 |
| 1:10642       | ["G","A"]                      | "rs558604819" | -1.00e+01 |
+---------------+--------------------------------+---------------+-----------+

+----------+----------------+----------------+----------+
| filters  | info.AF        | info.DR2       | info.IMP |
+----------+----------------+----------------+----------+
| set<str> | array<float64> | array<float64> |     bool |
+----------+----------------+----------------+----------+
| {}       | [4.46e-01]     | [0.00e+00]     |     True |
| {}       | [4.00e-04]     | [0.00e+00]     |     True |
| {}       | [4.73e-01]     | [0.00e+00]     |     True |
| {}       | [9.93e-01]     | [0.00e+00]     |     True |
| {}       | [3.70e-03]     | [0.00e+00]     |     True |
+----------+----------------+----------------+----------+
showing top 5 rows

mt には multi-allelic site が含まれます。次にその割合を確認してみます。

In [10]:
mt1 = mt.filter_rows(hl.len(mt.info.DR2) == 1)
mt1.count()

(2399023, 2318)

In [11]:
mtnot1 = mt.filter_rows(hl.len(mt.info.DR2) > 1)
mtnot1.count()

(29630, 2318)

1% ほどが multi-allelic site であることがわかります。
1% は無視するにはやや多いですが、このチュートリアルでは内容をわかりやすくするために除外して進めます。

下記のコードは、ゲノムデータのバリアント情報に `variantID` を追加します。

In [15]:
mt1 = mt1.annotate_rows(variantID = (hl.str(mt1.locus.contig) + ":" + hl.str(mt1.locus.position)) )

In [16]:
mt1.rows().show(5)

+---------------+--------------------------------+---------------+-----------+
| locus         | alleles                        | rsid          |      qual |
+---------------+--------------------------------+---------------+-----------+
| locus<GRCh37> | array<str>                     | str           |   float64 |
+---------------+--------------------------------+---------------+-----------+
| 1:10177       | ["A","AC"]                     | "rs367896724" | -1.00e+01 |
| 1:10235       | ["T","TA"]                     | "rs540431307" | -1.00e+01 |
| 1:10352       | ["T","TA"]                     | "rs555500075" | -1.00e+01 |
| 1:10616       | ["CCGCCGTTGCAAAGGCGCGCCG","C"] | "rs376342519" | -1.00e+01 |
| 1:10642       | ["G","A"]                      | "rs558604819" | -1.00e+01 |
+---------------+--------------------------------+---------------+-----------+

+----------+----------------+----------------+----------+-----------+
| filters  | info.AF        | info.DR2       | info.IMP | variantID |
+----------+----------------+----------------+----------+-----------+
| set<str> | array<float64> | array<float64> |     bool | str       |
+----------+----------------+----------------+----------+-----------+
| {}       | [4.46e-01]     | [0.00e+00]     |     True | "1:10177" |
| {}       | [4.00e-04]     | [0.00e+00]     |     True | "1:10235" |
| {}       | [4.73e-01]     | [0.00e+00]     |     True | "1:10352" |
| {}       | [9.93e-01]     | [0.00e+00]     |     True | "1:10616" |
| {}       | [3.70e-03]     | [0.00e+00]     |     True | "1:10642" |
+----------+----------------+----------------+----------+-----------+
showing top 5 rows

`variantID` のカラムが追加されていることが分かります。

# Step 4 imputation quality に基づいてゲノムデータをフィルタリングします
下記のコードでは、各バリアントの `imputation quality` の分布を表示します。 1分程度の待ち時間が生じます。

In [17]:
p = hl.plot.histogram(mt1.info.DR2.first(), title='Imputation Quality Histogram', legend='Imputation Quality (DR2)')
show(p)

`imputation quality` が低い（DR2 < 0.3）バリアントもいくつかあることが分かります。

下記のコードは、`imputation quality` が低い（DR2 < 0.3）バリアントを除外します。

In [19]:
mt1_filt = mt1.filter_rows(mt1.info.DR2.first()>=0.3)

下記のコードは、`imputation quality` が低い（DR2 < 0.3）バリアントを除外した後の分布を表示します。

In [21]:
p = hl.plot.histogram(mt1_filt.info.DR2.first(), title='Imputation Quality Histogram', legend='Imputation Quality (DR2)')
show(p)

`imputation quality` が低い（DR2 < 0.3）バリアントが除外されたことが分かります。

下記のコードは、`imputation quality` が低い（DR2 < 0.3）バリアントを除外したのちのバリアントの個数を表示します。

In [22]:
mt1_filt.count()

(2340844, 2318)

(2340844, 2318)
と表示されました。

これは、次のことを意味します。

- 研究対象者の人数が 2318 名
- バリアントの個数が 2340,844 個

下記のコードは、`imputation quality` が低い（DR2 < 0.3）バリアントを除外した後の allele frequency の分布を表示します。

In [24]:
p = hl.plot.histogram(mt1_filt.info.AF.first(), title='AF Histogram', legend='AF', bins=50)
show(p)

AF<1% のバリアントが多くあることが分かります。

# Step 5 PRSモデルを読み込みます
ここでは、PRSモデル `PGS000004` のみを読み込みます。

https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS000004/ScoringFiles/PGS000004.txt.gz をダウンロード、解凍後得られる `PGS000004.txt` を何らかのエディタで開き、先頭の#で始まるコメント行をすべて削除し、`prs-models` フォルダを作成し、その中に保存します。

その後 下記のコードを実行すると、`prs-models/PGS000004.txt` のデータが読み込まれます。

In [28]:
model_PGS000004 = hl.import_table('prs-models/PGS000004.txt', impute=True, force=True)

2022-09-13 20:09:40 Hail: INFO: Reading table to impute column types
2022-09-13 20:09:40 Hail: INFO: Finished type imputation
  Loading field 'chr_name' as type int32 (imputed)
  Loading field 'chr_position' as type int32 (imputed)
  Loading field 'effect_allele' as type str (imputed)
  Loading field 'other_allele' as type str (imputed)
  Loading field 'effect_weight' as type float64 (imputed)
  Loading field 'allelefrequency_effect' as type float64 (imputed)


下記のコードは、PRSモデルに含まれるバリアントの個数を表示します。

In [29]:
model_PGS000004.count()

313

`313` と表示されました。

これは、PRSモデルに含まれるバリアントの個数が 313 個であることを意味します。

# Step 6 PRSモデルに `variantID` を追加します
下記のコードは、読み込んだ PRS モデルの最初の 5 行（5 個のバリアントの情報）を表示します。

In [30]:
model_PGS000004.show(5)

,,,,,
chr_name,chr_position,effect_allele,other_allele,effect_weight,allelefrequency_effect
int32,int32,str,str,float64,float64
1,100880328,"""T""","""A""",3.73e-02,4.10e-01
1,10566215,"""G""","""A""",-5.86e-02,3.29e-01
1,110198129,"""C""","""CAAA""",4.58e-02,7.76e-01
1,114445880,"""A""","""G""",6.21e-02,1.66e-01
1,118141492,"""C""","""A""",4.52e-02,2.66e-01


下記のコードは、読み込んだ PRS モデルに `variantID` を追加します。

In [31]:
model_PGS000004 = model_PGS000004.annotate(
    variantID = hl.str(model_PGS000004.chr_name) + ":" + hl.str(model_PGS000004.chr_position) 
) 

`variantID` を追加したのちの最初の 5 行を表示します。

In [32]:
model_PGS000004.show(5)

,,,,,,
chr_name,chr_position,effect_allele,other_allele,effect_weight,allelefrequency_effect,variantID
int32,int32,str,str,float64,float64,str
1,100880328,"""T""","""A""",3.73e-02,4.10e-01,"""1:100880328"""
1,10566215,"""G""","""A""",-5.86e-02,3.29e-01,"""1:10566215"""
1,110198129,"""C""","""CAAA""",4.58e-02,7.76e-01,"""1:110198129"""
1,114445880,"""A""","""G""",6.21e-02,1.66e-01,"""1:114445880"""
1,118141492,"""C""","""A""",4.52e-02,2.66e-01,"""1:118141492"""


`variantID` のカラムが追加されていることが分かります。

# Step 7 ゲノムデータとPRSモデルに共通するバリアントを抽出します
下記のコードは、PRSモデルのバリアント情報を `variantID` で検索できるようにします。

In [33]:
model_PGS000004 = model_PGS000004.key_by('variantID')

下記のコードは、ゲノムデータと PRS モデルに共通するバリアントを抽出します。

In [37]:
mt1_filt.rows().show()

+---------------+------------+---------------+-----------+----------+
| locus         | alleles    | rsid          |      qual | filters  |
+---------------+------------+---------------+-----------+----------+
| locus<GRCh37> | array<str> | str           |   float64 | set<str> |
+---------------+------------+---------------+-----------+----------+
| 1:534247      | ["C","T"]  | "rs201475892" | -1.00e+01 | {}       |
| 1:565286      | ["C","T"]  | "rs1578391"   | -1.00e+01 | {}       |
| 1:674211      | ["C","T"]  | "rs546906063" | -1.00e+01 | {}       |
| 1:701299      | ["A","G"]  | "rs553919012" | -1.00e+01 | {}       |
| 1:701625      | ["T","C"]  | "rs576411494" | -1.00e+01 | {}       |
| 1:702958      | ["T","C"]  | "rs535793062" | -1.00e+01 | {}       |
| 1:703942      | ["G","C"]  | "rs548160064" | -1.00e+01 | {}       |
| 1:705452      | ["T","A"]  | "rs113340103" | -1.00e+01 | {}       |
| 1:705881      | ["C","T"]  | "rs116763968" | -1.00e+01 | {}       |
| 1:706357      | ["C","G"]  | "rs557777932" | -1.00e+01 | {}       |
+---------------+------------+---------------+-----------+----------+

+----------------+----------------+----------+------------+
| info.AF        | info.DR2       | info.IMP | variantID  |
+----------------+----------------+----------+------------+
| array<float64> | array<float64> |     bool | str        |
+----------------+----------------+----------+------------+
| [7.80e-03]     | [1.00e+00]     |    False | "1:534247" |
| [9.93e-01]     | [1.00e+00]     |    False | "1:565286" |
| [2.23e-02]     | [4.40e-01]     |     True | "1:674211" |
| [2.54e-02]     | [7.20e-01]     |     True | "1:701299" |
| [2.40e-03]     | [5.70e-01]     |     True | "1:701625" |
| [3.12e-02]     | [7.70e-01]     |     True | "1:702958" |
| [5.77e-02]     | [3.50e-01]     |     True | "1:703942" |
| [5.81e-02]     | [3.60e-01]     |     True | "1:705452" |
| [3.86e-02]     | [4.20e-01]     |     True | "1:705881" |
| [1.10e-03]     | [4.30e-01]     |     True | "1:706357" |
+----------------+----------------+----------+------------+
showing top 10 rows

In [38]:
mt_match = mt1_filt.annotate_rows(**model_PGS000004[mt1_filt.variantID])

In [39]:
mt_match = mt_match.filter_rows(hl.is_defined(mt_match.effect_weight))

下記のコードは、ゲノムデータと PRS モデルに共通するバリアントを表示します。 1分程度の待ち時間が生じます。

In [40]:
mt_match.rows().show()

2022-09-13 20:23:50 Hail: INFO: Ordering unsorted dataset with network shuffle7]
2022-09-13 20:23:51 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-09-13 20:24:05 Hail: INFO: Ordering unsorted dataset with network shuffle7]


+---------------+------------+---------------+-----------+----------+
| locus         | alleles    | rsid          |      qual | filters  |
+---------------+------------+---------------+-----------+----------+
| locus<GRCh37> | array<str> | str           |   float64 | set<str> |
+---------------+------------+---------------+-----------+----------+
| 1:7917076     | ["G","A"]  | "rs707475"    | -1.00e+01 | {}       |
| 1:10566215    | ["A","G"]  | "rs616488"    | -1.00e+01 | {}       |
| 1:18807339    | ["T","C"]  | "rs2992756"   | -1.00e+01 | {}       |
| 1:41380440    | ["C","T"]  | "rs4233486"   | -1.00e+01 | {}       |
| 1:41389220    | ["T","C"]  | "rs114282204" | -1.00e+01 | {}       |
| 1:46670206    | ["TC","T"] | "rs144105764" | -1.00e+01 | {}       |
| 1:51467096    | ["CT","C"] | "rs56168262"  | -1.00e+01 | {}       |
| 1:88156923    | ["G","A"]  | "rs17426269"  | -1.00e+01 | {}       |
| 1:88428199    | ["C","A"]  | "rs2151842"   | -1.00e+01 | {}       |
| 1:100880328   | ["A","T"]  | "rs612683"    | -1.00e+01 | {}       |
+---------------+------------+---------------+-----------+----------+

+----------------+----------------+----------+---------------+----------+
| info.AF        | info.DR2       | info.IMP | variantID     | chr_name |
+----------------+----------------+----------+---------------+----------+
| array<float64> | array<float64> |     bool | str           |    int32 |
+----------------+----------------+----------+---------------+----------+
| [3.36e-01]     | [1.00e+00]     |     True | "1:7917076"   |        1 |
| [3.00e-01]     | [1.00e+00]     |    False | "1:10566215"  |        1 |
| [5.89e-01]     | [1.00e+00]     |    False | "1:18807339"  |        1 |
| [6.72e-01]     | [1.00e+00]     |    False | "1:41380440"  |        1 |
| [9.10e-03]     | [9.80e-01]     |     True | "1:41389220"  |        1 |
| [1.34e-01]     | [9.90e-01]     |     True | "1:46670206"  |        1 |
| [4.25e-01]     | [9.70e-01]     |     True | "1:51467096"  |        1 |
| [7.98e-02]     | [1.00e+00]     |     True | "1:88156923"  |        1 |
| [1.79e-01]     | [1.00e+00]     |    False | "1:88428199"  |        1 |
| [4.00e-01]     | [9.90e-01]     |     True | "1:100880328" |        1 |
+----------------+----------------+----------+---------------+----------+

+--------------+---------------+--------------+---------------+
| chr_position | effect_allele | other_allele | effect_weight |
+--------------+---------------+--------------+---------------+
|        int32 | str           | str          |       float64 |
+--------------+---------------+--------------+---------------+
|      7917076 | "A"           | "G"          |     -4.09e-02 |
|     10566215 | "G"           | "A"          |     -5.86e-02 |
|     18807339 | "C"           | "T"          |     -5.64e-02 |
|     41380440 | "T"           | "C"          |      4.26e-02 |
|     41389220 | "C"           | "T"          |      1.55e-01 |
|     46670206 | "T"           | "TC"         |      4.47e-02 |
|     51467096 | "C"           | "CT"         |      3.74e-02 |
|     88156923 | "A"           | "G"          |      4.94e-02 |
|     88428199 | "A"           | "C"          |     -3.87e-02 |
|    100880328 | "T"           | "A"          |      3.73e-02 |
+--------------+---------------+--------------+---------------+

+------------------------+
| allelefrequency_effect |
+------------------------+
|                float64 |
+------------------------+
|               3.90e-01 |
|               3.29e-01 |
|               5.15e-01 |
|               6.44e-01 |
|               1.69e-02 |
|               2.97e-01 |
|               4.80e-01 |
|               1.49e-01 |
|               2.48e-01 |
|               4.10e-01 |
+------------------------+
showing top 10 rows

ゲノムデータと PRS モデルに共通するバリアントを抽出するために、`variantID` を利用しています。
`variantID` は、`染色体番号` と `塩基ポジション` から構成されます。

PRSを計算する際には、`染色体番号` と `塩基ポジション` だけでなく、`effect_allele` と `other_allele` もマッチさせる必要があります。 （そのための手順は後ほど解説します）

下記のコードは、ゲノムデータとPRSモデルに共通するバリアントを抽出した結果、ユニークな `variantID` が何個あるかをカウントします。 1分程度の待ち時間が生じます。

In [41]:
len(dict(mt_match.aggregate_rows(hl.agg.counter(mt_match.variantID))))

2022-09-13 20:31:32 Hail: INFO: Ordering unsorted dataset with network shuffle7]
2022-09-13 20:31:32 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-09-13 20:31:46 Hail: INFO: Ordering unsorted dataset with network shuffle7]


30

下記のコードは、PRSモデルのうち、1番染色体に位置するバリアントの個数を表示します。

In [42]:
model_PGS000004.filter(model_PGS000004.chr_name==1).count()

30

本チュートリアルでは、1番染色体のゲノムデータのみを用いています。
上記の結果から、PRSモデルの1番染色体バリアント 30 個のうち、30 個がゲノムデータに含まれていたことが分かります。

# Step 8 抽出したゲノムデータを保存します
今後のステップの実行時間を短縮するため、抽出したゲノムデータを保存し、再度読み込みます。

下記のコードは、抽出したゲノムデータを `outputs/chr1.beagle.matched.mt` に保存します。

In [43]:
mt_match.write('outputs/chr1.beagle.matched.mt', overwrite=True)

2022-09-13 20:42:55 Hail: INFO: Ordering unsorted dataset with network shuffle7]
2022-09-13 20:42:56 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-09-13 20:43:10 Hail: INFO: Ordering unsorted dataset with network shuffle7]
2022-09-13 20:43:46 Hail: INFO: wrote matrix table with 30 rows and 2318 columns in 7 partitions to outputs/chr1.beagle.matched.mt


下記のコードは、`outputs/chr1.beagle.matched.mt` を読み込みます。

In [44]:
mt_match = hl.read_matrix_table('outputs/chr1.beagle.matched.mt')

# Step 9 ゲノムデータとPRSモデルのアリル情報を照合します
下記のコードは、ゲノムデータのアリル情報とPRSモデルのアリル情報を比較し、合致しているかをチェックします。

ゲノムデータのアリル情報は、`mt_match.alleles[0]` と `mt_match.alleles[1]` に保存されています。 PRSモデルのアリル情報は、`mt_match.effect_allele` と `mt_match.other_allele` に保存されています。

`mt_match.alleles[0]` と `mt_match.effect_allele` が一致しており、かつ、`mt_match.alleles[1]` と `mt_match.other_allele` が一致している場合は、後ほど `allele flip` の操作が必要であるため、`flip` フラグを立てます。

`mt_match.alleles[1]` と `mt_match.effect_allele` が一致しており、かつ、`mt_match.alleles[0]` と `mt_match.other_allele` が一致している場合は、後ほど `allele flip` の操作が必要ないため、`flip` フラグを立てません。

In [46]:
flip = hl.case().when(
    (mt_match.effect_allele == mt_match.alleles[0])
    & (mt_match.other_allele == mt_match.alleles[1]), True ).when(
    (mt_match.effect_allele == mt_match.alleles[1])
    & (mt_match.other_allele == mt_match.alleles[0]), False ).or_missing()

In [47]:
mt_match = mt_match.annotate_rows( flip=flip )

下記のコードは、`flip` フラグ情報を表示します。

In [48]:
mt_match.rows().show(5)

+---------------+------------+---------------+-----------+----------+
| locus         | alleles    | rsid          |      qual | filters  |
+---------------+------------+---------------+-----------+----------+
| locus<GRCh37> | array<str> | str           |   float64 | set<str> |
+---------------+------------+---------------+-----------+----------+
| 1:7917076     | ["G","A"]  | "rs707475"    | -1.00e+01 | {}       |
| 1:10566215    | ["A","G"]  | "rs616488"    | -1.00e+01 | {}       |
| 1:18807339    | ["T","C"]  | "rs2992756"   | -1.00e+01 | {}       |
| 1:41380440    | ["C","T"]  | "rs4233486"   | -1.00e+01 | {}       |
| 1:41389220    | ["T","C"]  | "rs114282204" | -1.00e+01 | {}       |
+---------------+------------+---------------+-----------+----------+

+----------------+----------------+----------+--------------+----------+
| info.AF        | info.DR2       | info.IMP | variantID    | chr_name |
+----------------+----------------+----------+--------------+----------+
| array<float64> | array<float64> |     bool | str          |    int32 |
+----------------+----------------+----------+--------------+----------+
| [3.36e-01]     | [1.00e+00]     |     True | "1:7917076"  |        1 |
| [3.00e-01]     | [1.00e+00]     |    False | "1:10566215" |        1 |
| [5.89e-01]     | [1.00e+00]     |    False | "1:18807339" |        1 |
| [6.72e-01]     | [1.00e+00]     |    False | "1:41380440" |        1 |
| [9.10e-03]     | [9.80e-01]     |     True | "1:41389220" |        1 |
+----------------+----------------+----------+--------------+----------+

+--------------+---------------+--------------+---------------+
| chr_position | effect_allele | other_allele | effect_weight |
+--------------+---------------+--------------+---------------+
|        int32 | str           | str          |       float64 |
+--------------+---------------+--------------+---------------+
|      7917076 | "A"           | "G"          |     -4.09e-02 |
|     10566215 | "G"           | "A"          |     -5.86e-02 |
|     18807339 | "C"           | "T"          |     -5.64e-02 |
|     41380440 | "T"           | "C"          |      4.26e-02 |
|     41389220 | "C"           | "T"          |      1.55e-01 |
+--------------+---------------+--------------+---------------+

+------------------------+-------+
| allelefrequency_effect |  flip |
+------------------------+-------+
|                float64 |  bool |
+------------------------+-------+
|               3.90e-01 | False |
|               3.29e-01 | False |
|               5.15e-01 | False |
|               6.44e-01 | False |
|               1.69e-02 | False |
+------------------------+-------+
showing top 5 rows

- `flip` フラグが `False` の場合、`allele flip` の操作は必要ありません。
- `flip` フラグが `True` の場合、`allele flip` の操作が必要です。
- `flip` フラグが `NA` の場合、PRSモデルの `effect_allele` または `other_allele` がゲノムデータのアリル（`alleles[0]`および`alleles[1]`）と合致していないため、PRS計算時には考慮しません。

# Step 10 PRSを計算します
下記のコードは、各々のバリアントについて研究対象者の持っているアリル数（`mt_match.DS`）とバリアントの重み（`mt_match.effect_weight`）を掛け合わせて、ゲノムデータとPRSモデルの共通するバリアントについて足し合わせます。 これにより、PRSを計算することができます。

ここで、研究対象者の持っているアリル数を、下記のように計算しています。

- `flip` フラグが `False` の場合、`allele flip` の操作は必要ないため、`mt_match.DS` の値をアリル数として用います。
- `flip` フラグが `True` の場合、`allele flip` の操作が必要なため、`2 - mt_match.DS` の値をアリル数として用います。

In [68]:
prs=hl.agg.sum(hl.float64(mt_match.effect_weight) * hl.if_else(mt_match.flip, 2 - mt_match.DS.first(), mt_match.DS.first()))

In [69]:
mt_match = mt_match.annotate_cols(prs=prs)

下記のコードは、PRSの値を表示します。

In [70]:
mt_match.cols().show(5)

2022-09-13 21:11:34 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


,
s,prs
str,float64
"""_HG00096""",1.23e-01
"""_HG00097""",3.13e-01
"""_HG00098""",-7.49e-02
"""_HG00099""",1.61e-01
"""_HG00100""",4.59e-01


下記のコードは、PRSの分布を表示します。

In [71]:
p = hl.plot.histogram(mt_match.prs, title="PRS Histogram", legend="PGS000004", bins=20)
show(p)

# Step 11 PRSの計算結果を保存します

chr1.beagle.matched.mt')

下記のコードは、PRSの計算結果を `chr1.beagle.PGS000004.PRS.txt` ファイルに保存します。

In [72]:
mt_match.cols().export('chr1.beagle.PGS000004.PRS.txt')

2022-09-13 21:13:50 Hail: INFO: Coerced sorted dataset
2022-09-13 21:13:50 Hail: INFO: merging 17 files totalling 45.6K...
2022-09-13 21:13:50 Hail: INFO: while writing:
    chr1.beagle.PGS000004.PRS.txt
  merge time: 8.741ms


以上でこのチュートリアルは終了です。